# GoogleNet 구조

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

## conv layer function

In [2]:
def conv(n, size=(3,3), strides=(1,1), padding='same', name='conv'):
    output = layers.Conv2D(filters=n, kernel_size=size, strides=strides, padding=padding, activation='relu',name=name)
    return output

## inception module function

In [3]:
def inception_module(input, n_1x1, n_3x3r, n_3x3, n_5x5r, n_5x5, n_pool, idx_layer):
    name_layer = f'inception_{idx_layer}'

    conv_1x1 = conv(n=n_1x1, size=(1,1), padding='same', name=f'{name_layer}/1x1')(input)

    conv_3x3_reduce = conv(n=n_3x3r, size=(1,1), padding='same', name=f'{name_layer}/3x3_reduce')(input)
    conv_3x3 = conv(n=n_3x3, size=(3,3), padding='same', name=f'{name_layer}/3x3')(conv_3x3_reduce)

    conv_5x5_reduce = conv(n=n_5x5r, size=(1,1), padding='same', name=f'{name_layer}/5x5_reduce')(input)
    conv_5x5 = conv(n=n_5x5, size=(5,5), padding='same', name=f'{name_layer}/5x5')(conv_5x5_reduce)

    pool = layers.MaxPooling2D(pool_size=(3,3), strides=1, padding='same', name=f'{name_layer}/pool')(input)
    pool_proj = conv(n=n_pool, size=(1,1), padding='same', name=f'{name_layer}/pool_proj')(pool)

    # output : concat
    output = layers.Concatenate(axis=-1, name=f'{name_layer}/output')([conv_1x1,conv_3x3,conv_5x5,pool_proj])
    return output

## auxilary classifier function

In [4]:
def auxiliary_classifier(input, idx):
    # auxiliary classifier **********************
    avg_pool = layers.AveragePooling2D(pool_size=(5,5), strides=(3,3),
                                             name=f'aux{idx}/avg_pool')(input)
    conv = layers.Conv2D(128, (1,1), padding='same', activation='relu',
                               name=f'aux{idx}/conv')(avg_pool)
    flat = layers.Flatten()(conv)
    fc = layers.Dense(1024, activation='relu', name=f'aux{idx}/fc')(flat)
    drop_fc = layers.Dropout(rate=0.7)(fc)
    aux_classifier = layers.Dense(1000, activation='softmax', name=f'aux{idx}/classifier')(drop_fc)
    return aux_classifier

## Main modeling function

In [5]:
def create_googlenet():
    # creates GoogLeNet a.k.a. Inception v1 (Szegedy, 2015)
    input = tf.keras.Input(shape=(224, 224, 3))

    # 1 ---------------------------------------------------------------------
    conv1_7x7_s2 = conv(n=64, size=(7,7), strides=(2,2), padding='same', name='conv1/7x7_s2')(input)
    pool1 = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same', name='pool1/3x3_s2')(conv1_7x7_s2)
    pool1_norm1 = layers.Lambda(tf.nn.local_response_normalization, name='pool1/norm1')(pool1)

    # 2 ---------------------------------------------------------------------
    conv2_3x3_reduce = conv(n=64, size=(1,1), name='conv2/3x3_reduce')(pool1_norm1)
    conv2_3x3 = conv(n=192, size=(3,3), name='conv2/3x3')(conv2_3x3_reduce)
    conv2_norm2 = layers.Lambda(tf.nn.local_response_normalization, name='conv2/norm2')(conv2_3x3)
    pool2 = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same', name='pool2/3x3_s2')(conv2_norm2)

    # 3 ---------------------------------------------------------------------
    inception_3a = inception_module(input=pool2,
                                       n_1x1=64, n_3x3r=96, n_3x3=128,
                                       n_5x5r=16, n_5x5=32, n_pool=32, idx_layer='3a')
    inception_3b = inception_module(input=inception_3a,
                                        n_1x1=128, n_3x3r=128, n_3x3=192,
                                        n_5x5r=32, n_5x5=96, n_pool=64, idx_layer='3b')
    pool3 = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same', name='pool3/3x3_s2')(inception_3b)

    # 4 ---------------------------------------------------------------------
    inception_4a = inception_module(input=pool3,
                                        n_1x1=192, n_3x3r=96, n_3x3=208,
                                        n_5x5r=16, n_5x5=48, n_pool=64, idx_layer='4a')

    # auxilary classifier
    aux_classifier_1 = auxiliary_classifier(input=inception_4a, idx=1)

    inception_4b = inception_module(input=inception_4a,
                                        n_1x1=160, n_3x3r=112, n_3x3=224,
                                        n_5x5r=24, n_5x5=64, n_pool=64, idx_layer='4b')
    inception_4c = inception_module(input=inception_4b,
                                        n_1x1=128, n_3x3r=128, n_3x3=256,
                                        n_5x5r=24, n_5x5=64, n_pool=64, idx_layer='4c')
    inception_4d = inception_module(input=inception_4c,
                                        n_1x1=112, n_3x3r=144, n_3x3=288,
                                        n_5x5r=32, n_5x5=64, n_pool=64, idx_layer='4d')
    # auxilary classifier
    aux_classifier_2 = auxiliary_classifier(input=inception_4d, idx=2)

    inception_4e = inception_module(input=inception_4d,
                                        n_1x1=256, n_3x3r=160, n_3x3=320,
                                        n_5x5r=32, n_5x5=128, n_pool=128, idx_layer='4e')
    pool4 = layers.MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same', name='pool4/3x3_s2')(inception_4e)

    # 5 ---------------------------------------------------------------------
    inception_5a = inception_module(input=pool4,
                                        n_1x1=256, n_3x3r=160, n_3x3=320,
                                        n_5x5r=32, n_5x5=128, n_pool=128, idx_layer='5a')
    inception_5b = inception_module(input=inception_5a,
                                        n_1x1=384, n_3x3r=192, n_3x3=384,
                                        n_5x5r=48, n_5x5=128, n_pool=128, idx_layer='5b')


    avgpool_7x7 = layers.AveragePooling2D(pool_size=(7,7), strides=(1,1), name='pool5/7x7_s2')(inception_5b)

    # final(head) -----------------------------------------------------------
    flat = layers.Flatten()(avgpool_7x7)
    dropout = layers.Dropout(rate=0.4)(flat)
    main_classifier = layers.Dense(1000, activation='softmax', name='main_classifier')(dropout)

   # Model : input + output[aux.cls1, aux.cls2, main.cls]
    googlenet = tf.keras.Model(inputs=input, outputs=[aux_classifier_1, aux_classifier_2, main_classifier])

    return googlenet

In [6]:
model = create_googlenet()

ValueError: Argument `name` must be a string and cannot contain character `/`. Received: name=conv1/7x7_s2 (of type <class 'str'>)

In [ ]:
model.summary()